In [1]:
import sys
import os

def get_dir_n_levels_up(path, n):
    # Go up n levels from the given path
    for _ in range(n):
        path = os.path.dirname(path)
    return path

proj_root = get_dir_n_levels_up(os.path.abspath("__file__"), 4)
sys.path.append(proj_root)

import yaml
import torch
import numpy as np

from dqn.opinion_dynamics.opinion_dqn import AgentDQN
from dqn.opinion_dynamics.utils.my_logging import setup_logger
from dqn.opinion_dynamics.utils.experiment import build_environment

from dynamic_programming.opinion_dynamics.common.viz import plot_opinions_over_time, visualize_policy_from_env


def instantiate_agent(exp_subdir_path: str, train_env, validation_env) -> AgentDQN:
    """
    Instantiate an AgentDQN using the configuration stored in a YAML file 
    in the provided experiment subdirectory. The agent is created with the 
    given training and validation environments and loads its previous state.
    
    Args:
        exp_subdir_path (str): Path to the experiment subdirectory containing the config YAML and checkpoint files.
        train_env (gym.Env): The training environment instance.
        validation_env (gym.Env): The validation environment instance.
    
    Returns:
        AgentDQN: An instance of AgentDQN initialized using the experiment configuration and saved state.
    """
    # Assume the YAML configuration is stored as 'config.yaml' in the experiment folder.
    config_path = os.path.join(exp_subdir_path, "cfg.yaml")
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Config file not found at {config_path}")
    
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)
    
    # Instantiate the agent.
    # The resume_training_path is set to the experiment folder so that the agent loads saved weights/stats.
    agent = AgentDQN(
        train_env=train_env,
        validation_env=validation_env,
        resume_training_path=exp_subdir_path,
        experiment_name=config["experiment"],
        config=config,
        save_checkpoints=False,  # you can set this as needed
        logger=setup_logger("dqn")
    )
    
    return agent

def run_policy_agent(env, agent, max_steps=1000):
    """
    Run the simulation using the agent’s policy (exploitation only).
    
    Args:
        env: The environment (which must have a reset and step method).
        agent: An already-trained AgentDQN instance.
        max_steps: Maximum number of steps to run.
        
    Returns:
        opinions_over_time: Array of opinions (states) over time.
        time_points: Array of time stamps.
        rewards_over_time: Array of rewards collected at each step.
        actions_over_time: Array of actions taken at each step.
    """
    time_points = []
    rewards_over_time = []
    actions_over_time = []  # New: record the actions used.
    opinions_over_time = []
    
    current_time = 0.0
    # Reset environment
    state, _ = env.reset()
    opinions_over_time.append(state.copy())
    
    for step in range(max_steps):
        # Convert state to a batched tensor (batch size = 1)
        state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
        # Use the agent in exploitation mode (epsilon=0, random_action=False)
        # The agent.select_action returns (action, beta_idx, q_value)
        action, beta_idx, q_value = agent.select_action(state_tensor, epsilon=0.0, random_action=False)
        # action is returned as a NumPy array with shape (1, n_agents)
        action = np.squeeze(action)  # Now action has shape (n_agents,)
        actions_over_time.append(action.copy())
        
        # Apply the action in the environment.
        next_state, reward, done, truncated, _ = env.step(action, env.tau)
        opinions_over_time.append(next_state.copy())
        rewards_over_time.append(reward)
        time_points.append(current_time)
        
        current_time += env.tau
        state = next_state
        
        if done or truncated:
            print(f"Simulation ended at step {step}: done={done}, truncated={truncated}")
            break

    return (np.array(opinions_over_time),
            np.array(time_points),
            np.array(rewards_over_time),
            np.array(actions_over_time))

In [2]:
train_env = build_environment()
validation_env = build_environment()

exp_subdir = r"D:\Work\repos\RL\phd-rl-algos\dqn\opinion_dynamics\experiments\results\2025Mar21-013722_configs\0002_agent_params.args_.action_w_noise_amplitude_0__estimator.args_.lin_hidden_out_size_64\1"
agent = instantiate_agent(exp_subdir, train_env, validation_env)

num_steps = 100
states, times, rewards, actions = run_policy_agent(validation_env, agent)

plot_opinions_over_time(states)


2025-03-21 10:10:11,732 - dqn - INFO - opinion_dqn.py:224 - Loaded configuration settings.
2025-03-21 10:10:12,595 - dqn - INFO - opinion_dqn.py:283 - Initialized newtworks and optimizer.


d:\Work\repos\RL\phd-rl-algos\dqn\opinion_dynamics\opinion_dqn.py:302: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(training_stats_file)
d:\Work\rep

RuntimeError: Error(s) in loading state_dict for OpinionNet:
	size mismatch for predict_A_b_c.weight: copying a param with shape torch.Size([18, 64]) from checkpoint, the shape in current model is torch.Size([27, 64]).
	size mismatch for predict_A_b_c.bias: copying a param with shape torch.Size([18]) from checkpoint, the shape in current model is torch.Size([27]).

In [ ]:
actions

array([[0.12605561, 0.2739444 , 0.        , 0.        ],
       [0.07024676, 0.1626861 , 0.        , 0.16706714],
       [0.12027242, 0.27972758, 0.        , 0.        ],
       [0.07450724, 0.1610458 , 0.        , 0.16444695],
       [0.13524365, 0.26475632, 0.        , 0.        ],
       [0.16390488, 0.23609512, 0.        , 0.        ],
       [0.228904  , 0.17109601, 0.        , 0.        ],
       [0.4       , 0.        , 0.        , 0.        ],
       [0.15238687, 0.        , 0.        , 0.24761312],
       [0.4       , 0.        , 0.        , 0.        ],
       [0.4       , 0.        , 0.        , 0.        ],
       [0.09465958, 0.        , 0.        , 0.30534044],
       [0.3282587 , 0.03587065, 0.        , 0.        ],
       [0.27577245, 0.06211377, 0.        , 0.        ],
       [0.22703293, 0.08648355, 0.        , 0.        ],
       [0.070128  , 0.04021893, 0.        , 0.24943411],
       [0.22145991, 0.08927005, 0.        , 0.        ],
       [0.18087302, 0.10956349,

In [ ]:
gamma = 0.9
total_value = 0
for i, r in enumerate(rewards):
    total_value = total_value + (gamma**i) * r
    
total_value

-16.754519093908666